In [1]:
import io
import re
import numpy as np
import pandas as pd
import numpy.linalg as lin
import warnings 
from gensim.models import Word2Vec
import pickle
from sklearn.cluster import KMeans
warnings.filterwarnings('ignore')

In [2]:
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate

def t2h(text):
    return transliterate(text,sanscript.TELUGU,sanscript.DEVANAGARI)

In [3]:
stopwords=['కూడా', 'ఉన్నారు', 'ఎవరైనా', 'ఎప్పుడు', 'ఎక్కడైనా', 'అందుబాటులో', 'ఒక ప్రక్కన', 'సంబంధం', 'మాత్రమే', 'అడగండి', 'గా', 'మధ్య', 'ప్రకారం', 'అనుమతించు', 'మెచ్చుకో', 'వద్ద', 'తగిన', 'అడ్డంగా', 'అడగడం', 'ఇప్పటికే', 'కనిపిస్తాయి', 'కాదు', 'మరియు', 'మరొక', 'అనుగుణంగా', 'అయితే', 'చేయగలిగింది', 'దాదాపు', 'గురించి', 'పై', 'వెంట', 'వేరుగా', 'చుట్టూ', 'దూరంగా', 'ఏ', 'నిజంగా', 'అనుమతిస్తుంది', 'అందరూ', 'ఎవరో ఒకరు', 'వ్యతిరేకంగా', 'ఏదైనా', 'తర్వాత', 'మళ్ళీ', 'ఏమైనప్పటికి', 'ఒక']

In [4]:
vowels=set("అ ఆ ఇ ఈ ఉ ఊ ఋ ౠ ఌ ౡ ఎ ఏ ఐ ఒ ఓ ఔ అం అః".split())

conso="క	ఖ	గ	ఘ	ఙ	చ	ఛ	జ	ఝ	ఞ	ట	ఠ	డ	ఢ	ణ	త	థ	ద	ధ	న	ప	ఫ	బ	భ	మ	య	ర	ల	వ	శ	ష	స	హ	ళ	క్ష	ఱ"

matra = "ఀ	ఁ	ం	ః	ఄ	ఽ	ా	ి	ీ	ు	ూ	ృ	ౄ	ె	ే	ై	ొ	ో	ౌ	్	ౕ	ౖ	ౢ	ౣ"

halantha='్'

In [5]:
consonants=set(conso.split("\t"))
matra = set(matra.split("\t"))

vowels = list(vowels)
consonants=list(consonants)
matra=list(matra)

alphabets = vowels+consonants

In [6]:
delimiters=["।","?","!","."]

In [7]:
cbow=Word2Vec.load('cbow/te-d100-m2-cbow.model')

In [8]:
file=open('sentence.pkl','rb')
verse_embeddings_sentence=pickle.load(file)
file.close()
file=open('max.pkl','rb')
verse_embeddings_max=pickle.load(file)
file.close()
file=open('mean.pkl','rb')
verse_embeddings_mean=pickle.load(file)
file.close()

In [9]:
file=open('whole.pkl','rb')
verse_embeddings_whole=pickle.load(file)
file.close()

In [10]:
def remove_special_characters(text):
    string=''
    for i in text:
        if i in alphabets+matra+delimiters:
            string=string+i
        else:
            string=string+" "
    return " ".join(string.split())
def sent_tokenize(text):
    text=remove_special_characters(text)
    #regexPattern = '|'.join(map(re.escape, delimiters))
    regexPattern = '.'.join(map(re.escape, delimiters))
    sentences=re.split(regexPattern,text)
    return [sent.strip() for sent in sentences if len(sent.strip())>0]

In [11]:
def word_embedding(word):
    try:
        return cbow.wv[t2h(word)]
    except:
        return np.array([0.0]*100)

In [12]:
def cosine_similarity(a,b):
    try:
        return a.dot(b)/(lin.norm(a)*lin.norm(b))
    except:
        return -1.0

In [13]:
def ed(a,b):
    return lin.norm(a-b)

In [14]:
def sentence_embedding(sentence):
    sentence=remove_special_characters(sentence)
    words=sentence.split()
    words=[word  for word in words if word not in stopwords]
    if len(words)>0:
        sentence_embedding=[word_embedding(word) for word in words]
        return np.array(list(map(lambda x: sum(x)/len(x), zip(*sentence_embedding))))
    return np.array([0.0]*100)

In [15]:
data=pd.read_csv('../telugu.csv')

In [16]:
query='నాకు చాలా సమస్యలు ఉన్నాయి'
query_embedding=sentence_embedding(query)
query_embedding_list=query_embedding.reshape(1,-1).astype(np.float64)

# Max Strategy

## Cosine Similarity

In [17]:
def max_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_max]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [18]:
verse_index=max_cosine(query_embedding)

In [19]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
73,2,34,अकीर्तिं चापि भूतानि कथयिष्यन्ति तेऽव्ययाम्।\n...,గౌరవప్రదమైన వ్యక్తులకు సామాజిక ప్రతిష్ట చాలా మ...
596,17,9,कट्वम्ललवणात्युष्णतीक्ष्णरूक्षविदाहिनः।आहारा र...,ఎప్పుడైతే శాకాహార పదార్ధములను మితిమీరిన కారం/మ...
247,6,22,यं लब्ध्वा चापरं लाभं मन्यते नाधिकं ततः।\n\nयस...,"భౌతిక జగత్తులో ఎంత మేర సంపాదించినా, వ్యక్తి సం..."
640,18,25,अनुबन्धं क्षयं हिंसामनपेक्ष्य च पौरुषम्।मोहादा...,తమో-గుణములో ఉన్న జనుల బుద్ధి అజ్ఞానము చే ఆవరిం...
526,14,10,रजस्तमश्चाभिभूय सत्त्वं भवति भारत।रजः सत्त्वं ...,"ఒకే వ్యక్తి యొక్క ప్రవృత్తి, ఈ మూడు గుణములలో ఎ..."
75,2,36,अवाच्यवादांश्च बहून् वदिष्यन्ति तवाहिताः।\n\nन...,ఒకవేళ అర్జునుడు యుద్ధ భూమి నుండి పారిపోతే తోటి...
529,14,13,अप्रकाशोऽप्रवृत्तिश्च प्रमादो मोह एव च।तमस्येत...,"శ్రీ కృష్ణుడు మళ్ళీ ఒకసారి, త్రిగుణములు వ్యక్త..."
528,14,12,लोभः प्रवृत्तिरारम्भः कर्मणामशमः स्पृहा।रजस्ये...,"శ్రీ కృష్ణుడు మళ్ళీ ఒకసారి, త్రిగుణములు వ్యక్త..."
527,14,11,सर्वद्वारेषु देहेऽस्मिन्प्रकाश उपजायते।ज्ञानं ...,"శ్రీ కృష్ణుడు మళ్ళీ ఒకసారి, త్రిగుణములు వ్యక్త..."
485,13,4,तत्क्षेत्रं यच्च यादृक् च यद्विकारि यतश्च यत्।...,శ్రీ కృష్ణుడే తానే స్వయముగా ఇప్పుడు చాలా ప్రశ్...


## Euclidean Distance

In [20]:
def max_ed(query_embedding,counter=10):
    scoring=[ed(query_embedding,i) for i in verse_embeddings_max]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [21]:
verse_index=max_ed(query_embedding)

In [22]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
73,2,34,अकीर्तिं चापि भूतानि कथयिष्यन्ति तेऽव्ययाम्।\n...,గౌరవప్రదమైన వ్యక్తులకు సామాజిక ప్రతిష్ట చాలా మ...
589,17,2,श्री भगवानुवाचत्रिविधा भवति श्रद्धा देहिनां सा...,"ఎవ్వరూ కూడా శ్రద్ధ/విశ్వాసము లేకుండా ఉండరు, ఎం..."
75,2,36,अवाच्यवादांश्च बहून् वदिष्यन्ति तवाहिताः।\n\nन...,ఒకవేళ అర్జునుడు యుద్ధ భూమి నుండి పారిపోతే తోటి...
529,14,13,अप्रकाशोऽप्रवृत्तिश्च प्रमादो मोह एव च।तमस्येत...,"శ్రీ కృష్ణుడు మళ్ళీ ఒకసారి, త్రిగుణములు వ్యక్త..."
528,14,12,लोभः प्रवृत्तिरारम्भः कर्मणामशमः स्पृहा।रजस्ये...,"శ్రీ కృష్ణుడు మళ్ళీ ఒకసారి, త్రిగుణములు వ్యక్త..."
527,14,11,सर्वद्वारेषु देहेऽस्मिन्प्रकाश उपजायते।ज्ञानं ...,"శ్రీ కృష్ణుడు మళ్ళీ ఒకసారి, త్రిగుణములు వ్యక్త..."
640,18,25,अनुबन्धं क्षयं हिंसामनपेक्ष्य च पौरुषम्।मोहादा...,తమో-గుణములో ఉన్న జనుల బుద్ధి అజ్ఞానము చే ఆవరిం...
231,6,6,बन्धुरात्माऽऽत्मनस्तस्य येनात्मैवात्मना जितः।\...,"మనకు శత్రువులుగా అనిపించి, మనకు హాని చేయగలరేమో..."
660,18,45,स्वे स्वे कर्मण्यभिरतः संसिद्धिं लभते नरः।स्वक...,"స్వ-ధర్మ అంటే, మన గుణములు మరియు జీవిత స్థాయి (..."
596,17,9,कट्वम्ललवणात्युष्णतीक्ष्णरूक्षविदाहिनः।आहारा र...,ఎప్పుడైతే శాకాహార పదార్ధములను మితిమీరిన కారం/మ...


# Mean Pooling

## Cosine Similarity

In [23]:
def mean_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_mean]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [24]:
verse_index=mean_cosine(query_embedding)

In [25]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
73,2,34,अकीर्तिं चापि भूतानि कथयिष्यन्ति तेऽव्ययाम्।\n...,గౌరవప్రదమైన వ్యక్తులకు సామాజిక ప్రతిష్ట చాలా మ...
596,17,9,कट्वम्ललवणात्युष्णतीक्ष्णरूक्षविदाहिनः।आहारा र...,ఎప్పుడైతే శాకాహార పదార్ధములను మితిమీరిన కారం/మ...
247,6,22,यं लब्ध्वा चापरं लाभं मन्यते नाधिकं ततः।\n\nयस...,"భౌతిక జగత్తులో ఎంత మేర సంపాదించినా, వ్యక్తి సం..."
640,18,25,अनुबन्धं क्षयं हिंसामनपेक्ष्य च पौरुषम्।मोहादा...,తమో-గుణములో ఉన్న జనుల బుద్ధి అజ్ఞానము చే ఆవరిం...
526,14,10,रजस्तमश्चाभिभूय सत्त्वं भवति भारत।रजः सत्त्वं ...,"ఒకే వ్యక్తి యొక్క ప్రవృత్తి, ఈ మూడు గుణములలో ఎ..."
75,2,36,अवाच्यवादांश्च बहून् वदिष्यन्ति तवाहिताः।\n\nन...,ఒకవేళ అర్జునుడు యుద్ధ భూమి నుండి పారిపోతే తోటి...
529,14,13,अप्रकाशोऽप्रवृत्तिश्च प्रमादो मोह एव च।तमस्येत...,"శ్రీ కృష్ణుడు మళ్ళీ ఒకసారి, త్రిగుణములు వ్యక్త..."
528,14,12,लोभः प्रवृत्तिरारम्भः कर्मणामशमः स्पृहा।रजस्ये...,"శ్రీ కృష్ణుడు మళ్ళీ ఒకసారి, త్రిగుణములు వ్యక్త..."
527,14,11,सर्वद्वारेषु देहेऽस्मिन्प्रकाश उपजायते।ज्ञानं ...,"శ్రీ కృష్ణుడు మళ్ళీ ఒకసారి, త్రిగుణములు వ్యక్త..."
485,13,4,तत्क्षेत्रं यच्च यादृक् च यद्विकारि यतश्च यत्।...,శ్రీ కృష్ణుడే తానే స్వయముగా ఇప్పుడు చాలా ప్రశ్...


In [26]:
data.loc[227]["Commentary"]

'సన్యాసి అంటే మనో-ఇంద్రియముల సుఖాలను త్యజించిన వాడు అని. కానీ కేవలం సన్యసించటమే లక్ష్యం కాదు, ఇంకా, సన్యసించటం మాత్రమే లక్ష్యాన్ని చేరుకోవటానికి సరిపోదు. సన్యాసమంటే మనం తప్పు దిశలో పరిగెత్తటం ఆగినట్టు. మనం ఒకప్పుడు ఈ ప్రపంచంలో ఆనందాన్ని వెతుకుతూ ఉండే వాళ్ళం, కానీ భౌతిక సుఖాలలో ఆనందం లేదని అర్థం చేసుకున్నాము, కాబట్టి మనం ప్రపంచం దిశగా పరిగెత్తటం ఆపివేసాము. కానీ, ఆగిపోయినంత మాత్రాన లక్ష్యం చేరుకోలేము. ఆత్మ పొందవలసిన లక్ష్యం భగవత్-ప్రాప్తి. భగవంతుని దిశగా వెళ్ళటం - మనస్సుని ఆయన దిశగా తీసుకెళ్లటం - యోగ మార్గము. జీవిత లక్ష్యం యొక్క అసంపూర్ణ జ్ఞానం కలవారు, సన్యాసం తీసుకోవటమే అత్యున్నత లక్ష్యంగా అనుకుంటారు. నిజమైన జీవిత లక్ష్యాన్ని అర్థం చేసుకున్నవారు, భగవత్ ప్రాప్తియే తమ ఆధ్యాత్మిక ప్రయాణ అంతిమ లక్ష్యం గా తెలుసుకుంటారు.   5.4వ శ్లోకం యొక్క వివరణలో, రెండు రకాల సన్యాసం ఉంటాయి అని చెప్పబడింది - ఫాల్గు వైరాగ్యం మరియు యుక్త వైగాగ్యం అని. ఫాల్గు వైరాగ్యం అంటే ప్రాపంచిక వస్తువులను మాయకు చెందిన (భౌతిక శక్తి) వస్తువులగా పరిగణించి, ఆధ్యాత్మిక పురోగతికి అవరోధంగా ఉంటాయి కాబట్టి వాటిని త్యజించటం. యుక్త వ

## Euclidean Distance

In [27]:
def mean_ed(query_embedding,counter=10):
    scoring=[ed(query_embedding,i) for i in verse_embeddings_mean]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [28]:
verse_index=mean_ed(query_embedding)

In [29]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
73,2,34,अकीर्तिं चापि भूतानि कथयिष्यन्ति तेऽव्ययाम्।\n...,గౌరవప్రదమైన వ్యక్తులకు సామాజిక ప్రతిష్ట చాలా మ...
589,17,2,श्री भगवानुवाचत्रिविधा भवति श्रद्धा देहिनां सा...,"ఎవ్వరూ కూడా శ్రద్ధ/విశ్వాసము లేకుండా ఉండరు, ఎం..."
75,2,36,अवाच्यवादांश्च बहून् वदिष्यन्ति तवाहिताः।\n\nन...,ఒకవేళ అర్జునుడు యుద్ధ భూమి నుండి పారిపోతే తోటి...
529,14,13,अप्रकाशोऽप्रवृत्तिश्च प्रमादो मोह एव च।तमस्येत...,"శ్రీ కృష్ణుడు మళ్ళీ ఒకసారి, త్రిగుణములు వ్యక్త..."
528,14,12,लोभः प्रवृत्तिरारम्भः कर्मणामशमः स्पृहा।रजस्ये...,"శ్రీ కృష్ణుడు మళ్ళీ ఒకసారి, త్రిగుణములు వ్యక్త..."
527,14,11,सर्वद्वारेषु देहेऽस्मिन्प्रकाश उपजायते।ज्ञानं ...,"శ్రీ కృష్ణుడు మళ్ళీ ఒకసారి, త్రిగుణములు వ్యక్త..."
640,18,25,अनुबन्धं क्षयं हिंसामनपेक्ष्य च पौरुषम्।मोहादा...,తమో-గుణములో ఉన్న జనుల బుద్ధి అజ్ఞానము చే ఆవరిం...
231,6,6,बन्धुरात्माऽऽत्मनस्तस्य येनात्मैवात्मना जितः।\...,"మనకు శత్రువులుగా అనిపించి, మనకు హాని చేయగలరేమో..."
660,18,45,स्वे स्वे कर्मण्यभिरतः संसिद्धिं लभते नरः।स्वक...,"స్వ-ధర్మ అంటే, మన గుణములు మరియు జీవిత స్థాయి (..."
596,17,9,कट्वम्ललवणात्युष्णतीक्ष्णरूक्षविदाहिनः।आहारा र...,ఎప్పుడైతే శాకాహార పదార్ధములను మితిమీరిన కారం/మ...


# Line Matching

## Cosine Similarity

In [30]:
def line_cosine(query_embeddings,counter=10):
    scoring=[]
    for embeddings in verse_embeddings_sentence:
        score=[cosine_similarity(query_embedding,embed) for embed in embeddings]
        scoring.append(max(score))
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [31]:
verse_index=line_cosine(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
73,2,34,अकीर्तिं चापि भूतानि कथयिष्यन्ति तेऽव्ययाम्।\n...,గౌరవప్రదమైన వ్యక్తులకు సామాజిక ప్రతిష్ట చాలా మ...
596,17,9,कट्वम्ललवणात्युष्णतीक्ष्णरूक्षविदाहिनः।आहारा र...,ఎప్పుడైతే శాకాహార పదార్ధములను మితిమీరిన కారం/మ...
247,6,22,यं लब्ध्वा चापरं लाभं मन्यते नाधिकं ततः।\n\nयस...,"భౌతిక జగత్తులో ఎంత మేర సంపాదించినా, వ్యక్తి సం..."
640,18,25,अनुबन्धं क्षयं हिंसामनपेक्ष्य च पौरुषम्।मोहादा...,తమో-గుణములో ఉన్న జనుల బుద్ధి అజ్ఞానము చే ఆవరిం...
526,14,10,रजस्तमश्चाभिभूय सत्त्वं भवति भारत।रजः सत्त्वं ...,"ఒకే వ్యక్తి యొక్క ప్రవృత్తి, ఈ మూడు గుణములలో ఎ..."
75,2,36,अवाच्यवादांश्च बहून् वदिष्यन्ति तवाहिताः।\n\nन...,ఒకవేళ అర్జునుడు యుద్ధ భూమి నుండి పారిపోతే తోటి...
529,14,13,अप्रकाशोऽप्रवृत्तिश्च प्रमादो मोह एव च।तमस्येत...,"శ్రీ కృష్ణుడు మళ్ళీ ఒకసారి, త్రిగుణములు వ్యక్త..."
528,14,12,लोभः प्रवृत्तिरारम्भः कर्मणामशमः स्पृहा।रजस्ये...,"శ్రీ కృష్ణుడు మళ్ళీ ఒకసారి, త్రిగుణములు వ్యక్త..."
527,14,11,सर्वद्वारेषु देहेऽस्मिन्प्रकाश उपजायते।ज्ञानं ...,"శ్రీ కృష్ణుడు మళ్ళీ ఒకసారి, త్రిగుణములు వ్యక్త..."
485,13,4,तत्क्षेत्रं यच्च यादृक् च यद्विकारि यतश्च यत्।...,శ్రీ కృష్ణుడే తానే స్వయముగా ఇప్పుడు చాలా ప్రశ్...


## Euclidean Distance

In [32]:
def line_ed(query_embeddings,counter=10):
    scoring=[]
    for embeddings in verse_embeddings_sentence:
        score=[ed(query_embedding,embed) for embed in embeddings]
        scoring.append(max(score))
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [33]:
verse_index=line_ed(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
73,2,34,अकीर्तिं चापि भूतानि कथयिष्यन्ति तेऽव्ययाम्।\n...,గౌరవప్రదమైన వ్యక్తులకు సామాజిక ప్రతిష్ట చాలా మ...
589,17,2,श्री भगवानुवाचत्रिविधा भवति श्रद्धा देहिनां सा...,"ఎవ్వరూ కూడా శ్రద్ధ/విశ్వాసము లేకుండా ఉండరు, ఎం..."
75,2,36,अवाच्यवादांश्च बहून् वदिष्यन्ति तवाहिताः।\n\nन...,ఒకవేళ అర్జునుడు యుద్ధ భూమి నుండి పారిపోతే తోటి...
529,14,13,अप्रकाशोऽप्रवृत्तिश्च प्रमादो मोह एव च।तमस्येत...,"శ్రీ కృష్ణుడు మళ్ళీ ఒకసారి, త్రిగుణములు వ్యక్త..."
528,14,12,लोभः प्रवृत्तिरारम्भः कर्मणामशमः स्पृहा।रजस्ये...,"శ్రీ కృష్ణుడు మళ్ళీ ఒకసారి, త్రిగుణములు వ్యక్త..."
527,14,11,सर्वद्वारेषु देहेऽस्मिन्प्रकाश उपजायते।ज्ञानं ...,"శ్రీ కృష్ణుడు మళ్ళీ ఒకసారి, త్రిగుణములు వ్యక్త..."
640,18,25,अनुबन्धं क्षयं हिंसामनपेक्ष्य च पौरुषम्।मोहादा...,తమో-గుణములో ఉన్న జనుల బుద్ధి అజ్ఞానము చే ఆవరిం...
231,6,6,बन्धुरात्माऽऽत्मनस्तस्य येनात्मैवात्मना जितः।\...,"మనకు శత్రువులుగా అనిపించి, మనకు హాని చేయగలరేమో..."
660,18,45,स्वे स्वे कर्मण्यभिरतः संसिद्धिं लभते नरः।स्वक...,"స్వ-ధర్మ అంటే, మన గుణములు మరియు జీవిత స్థాయి (..."
596,17,9,कट्वम्ललवणात्युष्णतीक्ष्णरूक्षविदाहिनः।आहारा र...,ఎప్పుడైతే శాకాహార పదార్ధములను మితిమీరిన కారం/మ...


# Whole Part

## Cosine Similarity

In [34]:
def whole_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_whole]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [35]:
verse_index=whole_cosine(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
73,2,34,अकीर्तिं चापि भूतानि कथयिष्यन्ति तेऽव्ययाम्।\n...,గౌరవప్రదమైన వ్యక్తులకు సామాజిక ప్రతిష్ట చాలా మ...
596,17,9,कट्वम्ललवणात्युष्णतीक्ष्णरूक्षविदाहिनः।आहारा र...,ఎప్పుడైతే శాకాహార పదార్ధములను మితిమీరిన కారం/మ...
247,6,22,यं लब्ध्वा चापरं लाभं मन्यते नाधिकं ततः।\n\nयस...,"భౌతిక జగత్తులో ఎంత మేర సంపాదించినా, వ్యక్తి సం..."
640,18,25,अनुबन्धं क्षयं हिंसामनपेक्ष्य च पौरुषम्।मोहादा...,తమో-గుణములో ఉన్న జనుల బుద్ధి అజ్ఞానము చే ఆవరిం...
526,14,10,रजस्तमश्चाभिभूय सत्त्वं भवति भारत।रजः सत्त्वं ...,"ఒకే వ్యక్తి యొక్క ప్రవృత్తి, ఈ మూడు గుణములలో ఎ..."
75,2,36,अवाच्यवादांश्च बहून् वदिष्यन्ति तवाहिताः।\n\nन...,ఒకవేళ అర్జునుడు యుద్ధ భూమి నుండి పారిపోతే తోటి...
529,14,13,अप्रकाशोऽप्रवृत्तिश्च प्रमादो मोह एव च।तमस्येत...,"శ్రీ కృష్ణుడు మళ్ళీ ఒకసారి, త్రిగుణములు వ్యక్త..."
528,14,12,लोभः प्रवृत्तिरारम्भः कर्मणामशमः स्पृहा।रजस्ये...,"శ్రీ కృష్ణుడు మళ్ళీ ఒకసారి, త్రిగుణములు వ్యక్త..."
527,14,11,सर्वद्वारेषु देहेऽस्मिन्प्रकाश उपजायते।ज्ञानं ...,"శ్రీ కృష్ణుడు మళ్ళీ ఒకసారి, త్రిగుణములు వ్యక్త..."
485,13,4,तत्क्षेत्रं यच्च यादृक् च यद्विकारि यतश्च यत्।...,శ్రీ కృష్ణుడే తానే స్వయముగా ఇప్పుడు చాలా ప్రశ్...


## Euclidean Distance

In [36]:
def whole_ed(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_whole]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [37]:
verse_index=whole_ed(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
8,1,11,अयनेषु च सर्वेषु यथाभागमवस्थिताः।\n\nभीष्ममेवा...,"శత్రువులకు అసాధ్యుడైన భీష్ముడిని, తన సైన్యానిక..."
482,13,1,अर्जुन उवाच\n\nप्रकृतिं पुरुषं चैव क्षेत्रं क्...,"\n\n \n(*1) కొన్ని భగవద్గీత కోశములలో, ఈ శ్లో..."
419,11,13,तत्रैकस्थं जगत्कृत्स्नं प्रविभक्तमनेकधा।\n\nअप...,ఆ విశ్వ రూపములోని మహాద్భుతమైన ఆశ్చర్యములను వివ...
134,3,23,यदि ह्यहं न वर्तेयं जातु कर्मण्यतन्द्रितः।\n\n...,"ఈ భూలోకంలో తన దివ్య లీలలలో భాగంగా, శ్రీ కృష్ణు..."
100,2,61,तानि सर्वाणि संयम्य युक्त आसीत मत्परः।\n\nवशे ...,ఈ శ్లోకంలో 'యుక్తః' (కూడియున్న) అన్న పదం 'భక్త...
441,11,35,सञ्जय उवाच\n\nएतच्छ्रुत्वा वचनं केशवस्य\n\nकृत...,"ఇక్కడ, అర్జునుడు ""కిరీటి"" (కిరీటధారి) అని సంభో..."
680,18,65,मन्मना भव मद्भक्तो मद्याजी मां नमस्कुरु।मामेवै...,"9వ అధ్యాయములో, శ్రీ కృష్ణుడు అర్జునుడికి అత్యం..."
449,11,43,पितासि लोकस्य चराचरस्य\n\nत्वमस्य पूज्यश्च गुर...,శ్రీ కృష్ణుడే సర్వోన్నతుడు మరియు అందరికంటే పెద...
12,1,15,पाञ्चजन्यं हृषीकेशो देवदत्तं धनंजयः।\n\nपौण्ड्...,"శ్రీ కృష్ణుడికి వాడబడిన 'హృషీకేశః' అన్న పదం, ఆ..."
130,3,19,तस्मादसक्तः सततं कार्यं कर्म समाचर।\n\nअसक्तो ...,"3.8వ శ్లోకం నుండి 3.16వ శ్లోకం వరకు, శ్రీ కృష్..."


# Clustering

## Max Pooling

In [38]:
kmeans_max = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_max)

In [39]:
def kmeans_max_cluster(query_embedding_list):
    label=kmeans_max.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_max.labels_[i]==label]
    return verse_index

In [40]:
verse_index=kmeans_max_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
485,13,4,तत्क्षेत्रं यच्च यादृक् च यद्विकारि यतश्च यत्।...,శ్రీ కృష్ణుడే తానే స్వయముగా ఇప్పుడు చాలా ప్రశ్...


## Mean Pooling

In [41]:
kmeans_mean = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_mean)

In [42]:
def kmeans_mean_cluster(query_embedding_list):
    label=kmeans_mean.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_mean.labels_[i]==label]
    return verse_index

In [43]:
verse_index=kmeans_mean_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
485,13,4,तत्क्षेत्रं यच्च यादृक् च यद्विकारि यतश्च यत्।...,శ్రీ కృష్ణుడే తానే స్వయముగా ఇప్పుడు చాలా ప్రశ్...


## Whole

In [44]:
kmeans_whole = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_whole)

In [45]:
def kmeans_whole_cluster(query_embedding_list):
    label=kmeans_whole.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_whole.labels_[i]==label]
    return verse_index

In [46]:
verse_index=kmeans_whole_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
485,13,4,तत्क्षेत्रं यच्च यादृक् च यद्विकारि यतश्च यत्।...,శ్రీ కృష్ణుడే తానే స్వయముగా ఇప్పుడు చాలా ప్రశ్...
